GOOGLE DRIVE CONNECT

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


BASIC IMPORTS

In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime


PROJECT ROOT DEFINE

In [3]:
# Project Root
PROJECT_ROOT = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN"

# Important Paths
MODEL_PATH = os.path.join(PROJECT_ROOT, "03_Models", "bone_fracture_model_phase1.h5")
UPLOAD_DIR = os.path.join(PROJECT_ROOT, "04_Inference", "uploads")
REPORT_DIR = os.path.join(PROJECT_ROOT, "06_Reports")

# Create folders if not exist (safe)
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

print("✅ Project paths configured")
print("Model:", MODEL_PATH)
print("Uploads:", UPLOAD_DIR)
print("Reports:", REPORT_DIR)


✅ Project paths configured
Model: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/03_Models/bone_fracture_model_phase1.h5
Uploads: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/04_Inference/uploads
Reports: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports




MODEL LOAD

In [4]:
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully")


✅ Model loaded successfully


IMAGE PREPROCESSING FUNCTION

In [5]:
from tensorflow.keras.preprocessing import image

IMG_SIZE = (224, 224)

def load_and_preprocess_image(img_path):
    """
    Load image from path and preprocess for model prediction
    """
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


CENTRAL PREDICTION FUNCTION

In [6]:
def predict_fracture(img_path, model):
    """
    Predict fracture from X-ray image
    """
    img_array = load_and_preprocess_image(img_path)

    prediction = model.predict(img_array)[0][0]

    label = "Fracture Detected" if prediction >= 0.5 else "No Fracture"
    confidence = float(prediction if prediction >= 0.5 else 1 - prediction)

    return {
        "label": label,
        "confidence": round(confidence * 100, 2)
    }


QUICK SANITY TEST

In [7]:
# Test image path (apna koi bhi test X-ray)
TEST_IMAGE = os.path.join(
    PROJECT_ROOT,
    "01_Data/raw/bone fracture dataset/test/fractured",
    os.listdir(os.path.join(PROJECT_ROOT, "01_Data/raw/bone fracture dataset/test/fractured"))[0]
)

result = predict_fracture(TEST_IMAGE, model)
print("✅ Prediction Result:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
✅ Prediction Result: {'label': 'No Fracture', 'confidence': 87.22}


MEDICAL LOGIC ENGINE

In [8]:
def generate_medical_insight(prediction_result):
    """
    Convert model output into patient-friendly medical insight
    """
    label = prediction_result["label"]
    confidence = prediction_result["confidence"]

    if label == "Fracture Detected":
        fracture_type = "Possible Bone Fracture"
        severity = "Moderate to High Risk"
        advice = [
            "Avoid movement of the affected area",
            "Consult an orthopedic doctor immediately",
            "X-ray or CT confirmation is recommended",
            "Use temporary immobilization if advised"
        ]
    else:
        fracture_type = "No Visible Fracture"
        severity = "Low Risk"
        advice = [
            "No immediate fracture detected",
            "If pain persists, consult a doctor",
            "Avoid heavy strain for a few days",
            "Follow basic rest and care"
        ]

    return {
        "diagnosis": fracture_type,
        "risk_level": severity,
        "confidence": confidence,
        "recommendations": advice
    }


FINAL PATIENT OUTPUT FORMATTER

In [9]:
def build_patient_report(img_path, model):
    """
    Full inference pipeline for patient output
    """
    prediction = predict_fracture(img_path, model)
    medical_insight = generate_medical_insight(prediction)

    report = {
        "patient_note": "This is an AI-assisted analysis, not a medical diagnosis.",
        "result": medical_insight,
        "copyright": "© SHUBHAM MADDHESIYA",
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return report


LIVE TEST

In [10]:
final_report = build_patient_report(TEST_IMAGE, model)

from pprint import pprint
pprint(final_report)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
{'copyright': '© SHUBHAM MADDHESIYA',
 'patient_note': 'This is an AI-assisted analysis, not a medical diagnosis.',
 'result': {'confidence': 87.22,
            'diagnosis': 'No Visible Fracture',
            'recommendations': ['No immediate fracture detected',
                                'If pain persists, consult a doctor',
                                'Avoid heavy strain for a few days',
                                'Follow basic rest and care'],
            'risk_level': 'Low Risk'},
 'timestamp': '2026-02-03 14:52:37'}


REQUIRED IMPORTS

In [12]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.0 MB/s eta 0:00:00


In [13]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.lib.utils import ImageReader
from datetime import datetime
import os


FIXED PDF SAVE PATH

In [14]:
PDF_DIR = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports"
os.makedirs(PDF_DIR, exist_ok=True)


CORE PDF GENERATOR FUNCTION

In [15]:
def generate_pdf_report(report_data, image_path):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_path = f"{PDF_DIR}/Bone_Fracture_Report_{timestamp}.pdf"

    c = canvas.Canvas(pdf_path, pagesize=A4)
    width, height = A4

    # Title
    c.setFont("Helvetica-Bold", 18)
    c.drawCentredString(width / 2, height - 2*cm, "Bone Fracture Analysis Report")

    # Subtitle
    c.setFont("Helvetica", 10)
    c.drawCentredString(width / 2, height - 2.8*cm, "AI-assisted Medical Insight")

    y = height - 4*cm

    # Image
    try:
        img = ImageReader(image_path)
        c.drawImage(img, 2*cm, y - 10*cm, width=12*cm, height=10*cm, preserveAspectRatio=True)
    except:
        c.drawString(2*cm, y, "X-ray image not available")

    y -= 11*cm

    result = report_data["result"]

    # Report content
    c.setFont("Helvetica", 11)
    c.drawString(2*cm, y, f"Diagnosis: {result['diagnosis']}")
    y -= 1*cm

    c.drawString(2*cm, y, f"Risk Level: {result['risk_level']}")
    y -= 1*cm

    c.drawString(2*cm, y, f"Confidence: {result['confidence']}%")
    y -= 1.2*cm

    c.setFont("Helvetica-Bold", 11)
    c.drawString(2*cm, y, "Recommendations:")
    y -= 0.8*cm

    c.setFont("Helvetica", 11)
    for advice in result["recommendations"]:
        c.drawString(2.5*cm, y, f"- {advice}")
        y -= 0.7*cm

    # Footer
    c.setFont("Helvetica-Oblique", 9)
    c.drawString(2*cm, 2*cm, report_data["patient_note"])
    c.drawRightString(width - 2*cm, 2*cm, "© SHUBHAM MADDHESIYA")

    c.save()
    return pdf_path


ONE-CLICK PDF GENERATION

In [16]:
pdf_file = generate_pdf_report(final_report, TEST_IMAGE)
print("✅ PDF Report Generated:")
print(pdf_file)


✅ PDF Report Generated:
/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports/Bone_Fracture_Report_20260203_150149.pdf


others...

In [17]:
%%writefile /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py


Writing /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py


In [18]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.5 MB/s eta 0:00:00


for .py file overright

In [21]:
%%writefile /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py
%%writefile /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py

import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

st.set_page_config(page_title="Bone Fracture Detection", layout="centered")

st.title("🦴 Bone Fracture Detection System")
st.caption("© SHUBHAM MADDHESIYA")

MODEL_PATH = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/03_Models/bone_fracture_model_phase1.h5"
model = tf.keras.models.load_model(MODEL_PATH)

uploaded_file = st.file_uploader("Upload X-ray Image", type=["jpg", "png", "jpeg"])

def generate_pdf(result, save_path):
    c = canvas.Canvas(save_path, pagesize=A4)
    c.setFont("Helvetica", 12)
    c.drawString(2*cm, 27*cm, "Bone Fracture Detection Report")
    c.drawString(2*cm, 26*cm, f"Result: {result}")
    c.drawString(2*cm, 25*cm, "© SHUBHAM MADDHESIYA")
    c.save()

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded X-ray", use_column_width=True)

    img = image.resize((224, 224))
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    prediction = model.predict(img_array)[0][0]
    result = "Fractured" if prediction > 0.5 else "Normal"

    st.subheader(f"🧾 Prediction: {result}")

    if st.button("📄 Download PDF Report"):
        os.makedirs("/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports", exist_ok=True)
        filename = f"Bone_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
        path = f"/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports/{filename}"
        generate_pdf(result, path)
        st.success("PDF Generated Successfully")


Overwriting /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py


for .py file show

In [22]:
!sed -n '1,40p' /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py


%%writefile /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py

import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

st.set_page_config(page_title="Bone Fracture Detection", layout="centered")

st.title("🦴 Bone Fracture Detection System")
st.caption("© SHUBHAM MADDHESIYA")

MODEL_PATH = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/03_Models/bone_fracture_model_phase1.h5"
model = tf.keras.models.load_model(MODEL_PATH)

uploaded_file = st.file_uploader("Upload X-ray Image", type=["jpg", "png", "jpeg"])

def generate_pdf(result, save_path):
    c = canvas.Canvas(save_path, pagesize=A4)
    c.setFont("Helvetica", 12)
    c.drawString(2*cm, 27*cm, "Bone Fracture Detection Report")
    c.drawString(2*cm, 26*cm, f"Result: {result}")
    c.drawString(2*cm, 25*cm, "© SHUBHAM MADDHESIYA")
    c.save()

if uploaded_file:
    ima

In [23]:
!pip install streamlit


for Streamlit app run

In [24]:
!streamlit run /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_UI/app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://136.118.151.114:8501

  Stopping...
  Stopping...
